In [ ]:
import os
os.environ["XLA_FLAGS"] = "--xla_force_host_platform_device_count=8"

import functools

import flax.linen as nn
import jax
import jax.numpy as jnp
import numpy as np
import optax
from jax.experimental.shard_map import shard_map
from jax.sharding import Mesh
from jax.sharding import PartitionSpec as P
from ml_collections import ConfigDict

In [66]:
device_array = np.array(jax.devices())
mesh = Mesh(device_array, ("x",))
mesh

Mesh(device_ids=array([0, 1, 2, 3, 4, 5, 6, 7]), axis_names=('x',), axis_types=(Auto,))

In [67]:
device_array

array([CpuDevice(id=0), CpuDevice(id=1), CpuDevice(id=2), CpuDevice(id=3),
       CpuDevice(id=4), CpuDevice(id=5), CpuDevice(id=6), CpuDevice(id=7)],
      dtype=object)

In [77]:
class ShardedDense(nn.Module):
    features: int = 64
    kernel_init: nn.initializers.Initializer = nn.initializers.lecun_normal()
    bias_init: nn.initializers.Initializer = nn.initializers.zeros
    use_bias: bool = True
    tp_axis_name: str = 'tp'

    @nn.compact
    def __call__(self, x):
        input_dim = x[0]
        weights_sharding = P(None, self.tp_axis_name)
        weights = self.param(
                'weights', 
                self.kernel_init, 
                (input_dim, self.features), 
                jax.default_prng_key(),
                _param_axes=nn.split_axis('hidden', 0, self.tp_axis_name)
            )
        if self.use_bias:
            bias_sharding = P(None, self.tp_axis_name)
            bias = self.param(
                    'bias',
                    self.bias_init,
                    (self.features,),
                    jax.default_prng_key(),
                    _param_axes=nn.split_axis('hidden', 0, self.tp_axis_name)
            )
        else:
            bias = None
        
        y_local = x @ weights
        y_local = y_local + bias if bias is not None else y_local
        y_global = jax.lax.all_gather(y_local, self.tp_axis_name, axis=1, tiled=True)

        return y_global

In [78]:
key = jax.random.PRNGKey(0)
input_shape = (8, 32)
output_features = 64

sharded_dense_layer = ShardedDense(features=output_features)

In [79]:
#shard x upon initlization
x = jax.numpy.ones(input_shape)
input_sharding = jax.sharding.NamedSharding(mesh, P('x',))
sharded_input = jax.device_put(x, input_sharding)

In [80]:
@jax.jit
def init_params(key, input):
    variables = sharded_dense_layer.init(key, input)
    return variables['params']

params = init_params(key, sharded_input)

AttributeError: module 'jax' has no attribute 'default_prng_key'